## basics

In [1]:
import numpy as np

In [2]:
a = np.random.randn(5)
a

array([-1.41130058,  0.10873048, -1.02421963, -1.02447402,  0.64422318])

In [3]:
np.dot(a, a.T)

4.517188007088658

In [4]:
# (N, ) - rank one array, non-intuitive structure, neither a column or a row vector, does not behave consistenly
# For all programming, strictly do not use rank 1 array, always prefer a row (1,n) or column(n,1) vector
a.shape 

(5,)

In [5]:
b = np.random.randn(5,1) # shape: 5x1
b

array([[ 0.61292059],
       [-0.51108943],
       [ 0.1612061 ],
       [ 1.13481523],
       [ 0.39003849]])

In [6]:
np.dot(b,b.T)

array([[ 0.37567165, -0.31325723,  0.09880654,  0.69555162,  0.23906262],
       [-0.31325723,  0.2612124 , -0.08239073, -0.57999206, -0.19934455],
       [ 0.09880654, -0.08239073,  0.02598741,  0.18293914,  0.06287658],
       [ 0.69555162, -0.57999206,  0.18293914,  1.28780559,  0.44262162],
       [ 0.23906262, -0.19934455,  0.06287658,  0.44262162,  0.15213002]])

In [7]:
b.shape

(5, 1)

In [8]:
w = np.arange(5)
w

array([0, 1, 2, 3, 4])

In [9]:
w.shape # rank 1 array

(5,)

In [10]:
# reshape
w1 = w.reshape((5,1))
w1

array([[0],
       [1],
       [2],
       [3],
       [4]])

In [11]:
w2 = w.reshape([-1, 1]) # 1 column with flexible number of rows
w2

array([[0],
       [1],
       [2],
       [3],
       [4]])

In [12]:
wt = np.arange(10)
wt = wt.reshape([-1,1])
wt

array([[0],
       [1],
       [2],
       [3],
       [4],
       [5],
       [6],
       [7],
       [8],
       [9]])

In [13]:
w3 = w[:, np.newaxis] # column vector
w3

array([[0],
       [1],
       [2],
       [3],
       [4]])

In [14]:
w4 = w[np.newaxis, :] # row vector
w4

array([[0, 1, 2, 3, 4]])

In [15]:
w4.shape

(1, 5)

In [16]:
# flatten back to rank 1 array
w4.flatten().shape

(5,)

## MPT / MVP

In [17]:
# import libraries
from numpy import *
from numpy.linalg import multi_dot
import pandas as pd
import yfinance as yf

import cufflinks as cf
cf.set_config_file(offline=True, dimensions=((1000,600)), theme='henanigans')

import plotly.express as px # for Efficient Frontier plot
px.defaults.template, px.defaults.width, px.defaults.height = 'plotly_dark', 1000, 600

import warnings
warnings.filterwarnings('ignore')

In [18]:
# stock list
symbols = ['AMD','CSCO','INTC','INTU','NVDA']

# num of assets
numofasset = len(symbols)

# num of portfolios for simulation / optimization
numofportfolio = 5000

In [19]:
# fetch data from yf
nasdaqstocks = yf.download(symbols, start='2017-01-01', end='2021-12-31', progress=False)['Adj Close']
nasdaqstocks.to_csv('data/nasdaqstocks.csv')

In [20]:
# read local stored data
df = pd.read_csv('data/nasdaqstocks.csv', index_col=0, parse_dates=True)
df

,AMD,CSCO,INTC,INTU,NVDA
Date,,,,,
2017-01-03,11.430000,25.985624,31.972996,110.028534,25.166452
2017-01-04,11.430000,25.831152,31.807026,110.249054,25.753609
2017-01-05,11.240000,25.891224,31.754602,110.929848,25.099833
2017-01-06,11.320000,25.942715,31.868170,112.381989,25.435356
2017-01-09,11.490000,25.899809,31.981731,112.045372,26.466589
...,...,...,...,...,...
2021-12-23,146.139999,61.915154,50.922092,634.972839,296.349487
2021-12-27,154.360001,63.048477,51.547329,652.013062,309.397308
2021-12-28,153.149994,63.157833,51.368690,649.026550,303.168335


In [21]:
# visualize price history (nomralized price history)
df['2021':].normalize().iplot(kind='line')

In [22]:
# calculate returns
returns = df.pct_change().fillna(0)
returns

,AMD,CSCO,INTC,INTU,NVDA
Date,,,,,
2017-01-03,0.000000,0.000000,0.000000,0.000000,0.000000
2017-01-04,0.000000,-0.005945,-0.005191,0.002004,0.023331
2017-01-05,-0.016623,0.002326,-0.001648,0.006175,-0.025386
2017-01-06,0.007117,0.001989,0.003576,0.013091,0.013368
2017-01-09,0.015018,-0.001654,0.003563,-0.002995,0.040543
...,...,...,...,...,...
2021-12-23,0.015707,0.012189,0.006671,0.006412,0.008163
2021-12-27,0.056247,0.018304,0.012278,0.026836,0.044028
2021-12-28,-0.007839,0.001734,-0.003466,-0.004580,-0.020133


In [23]:
# plot annualized return and volatility
pd.DataFrame({
    'Annualized Return': round(returns.mean()*252*100,2),
    'Annualized Volatility': round(returns.std()*sqrt(252)*100,2)
}).iplot(kind='bar', shared_xaxes=True, subplots=True)

## portfolio statistics

In [24]:
# define weights for equal weighted portfolio
wts = numofasset * [1./numofasset]
wts

[0.2, 0.2, 0.2, 0.2, 0.2]

In [25]:
# reshape wts to column vector
wts = array(wts)[:, newaxis]
wts

array([[0.2],
       [0.2],
       [0.2],
       [0.2],
       [0.2]])

In [26]:
wts.shape

(5, 1)

In [27]:
# calculate portfolio returns
ret = array(returns.mean()*252)[:,newaxis]
ret

array([[0.66721301],
       [0.21500209],
       [0.15552522],
       [0.40136363],
       [0.60531439]])

In [28]:
ret.shape

(5, 1)

In [29]:
# portfolio returns using @ (matrix multiply)
wts.T@ret

array([[0.40888367]])

In [30]:
# portfolio volatility using multi_dot
cov = returns.cov()*252
var = multi_dot([wts.T, cov, wts])
sqrt(var)

array([[0.30735395]])

## portfolio simulation

In [31]:
def portfolio_simulation(returns):
    
    # initialize the lists
    rets = []; vols = []; wts = []
    
    # simulate 5000 portfolios
    for i in range(numofportfolio):
        # generate random weights
        weights = random.random(numofasset)[:, newaxis]
        # set weights such that sum of weights equals to 1
        weights /= sum(weights)
        
        # portfolio statistics
        rets.append(weights.T @ array(returns.mean()*252)[:,newaxis]) 
        vols.append(sqrt(multi_dot([weights.T, returns.cov()*252, weights]))) 
        wts.append(weights.flatten()) 
    
    # create a dataframe for analysis
    portdf = 100* pd.DataFrame({
        'port_rets': array(rets).flatten(),
        'port_vols': array(vols).flatten(),
        'weights': list(array(wts))
    })
                    
    portdf['sharpe_ratio'] = portdf['port_rets'] / portdf['port_vols']
                    
    return round(portdf,2)

In [32]:
temp = portfolio_simulation(returns)
temp

,port_rets,port_vols,weights,sharpe_ratio
0,38.89,32.04,"[32.11848505007692, 18.345677696790943, 32.142...",1.21
1,34.34,28.42,"[7.933287804557235, 6.819977819853343, 33.2898...",1.21
2,38.74,32.12,"[31.771597539970074, 8.782464824752882, 37.306...",1.21
3,43.64,31.92,"[37.131439427016815, 35.74376937773221, 1.0116...",1.37
4,44.27,33.14,"[18.955799752156636, 17.8574562136136, 17.9996...",1.34
...,...,...,...,...
4995,34.16,27.23,"[2.738591232048439, 31.35598142881109, 17.2818...",1.25
4996,46.03,34.50,"[20.570376379383067, 25.770629301402593, 11.81...",1.33
4997,36.84,29.15,"[14.835668753006265, 14.271035576319202, 29.09...",1.26
4998,38.99,28.41,"[11.12106706271445, 34.26255580804489, 6.47760...",1.37


In [33]:
# get the location id of that with maximum sharpe ratio
temp.sharpe_ratio.idxmax()
temp.iloc[temp.sharpe_ratio.idxmax()]

port_rets                                                   50.11
port_vols                                                   33.69
weights         [25.253216148289088, 7.146658145567948, 0.7814...
sharpe_ratio                                                 1.49
Name: 1134, dtype: object

In [34]:
# verify above result
temp.describe().T

,count,mean,std,min,25%,50%,75%,max
port_rets,5000.0,40.862550,5.810842,22.42,37.1400,40.78,44.5600,61.19
port_vols,5000.0,31.463890,2.881559,25.39,29.4575,31.18,33.0425,49.52
sharpe_ratio,5000.0,1.294686,0.100824,0.77,1.2400,1.31,1.3700,1.49


In [35]:
# max sharpe ratio portfolio weights
msrpwts = temp['weights'][temp['sharpe_ratio'].idxmax()]

# allocation to achieve max sharpe ratio
dict(zip(symbols, msrpwts))

{'AMD': 25.253216148289088,
 'CSCO': 7.146658145567948,
 'INTC': 0.7814991396707743,
 'INTU': 43.376890667869525,
 'NVDA': 23.441735898602666}

## visualization

In [36]:
# plot simulated portfolio
fig = px.scatter(
    temp, x='port_vols', y='port_rets', color='sharpe_ratio',
    labels = {'port_vols': 'expected volatility', 'port_rets':'expected return', 'sharpe_ratio': 'sharpe ratio'},
    title = "Monte Carlo Simulated Portfolio"
    ).update_traces(mode='markers', marker = dict(symbol = 'cross'))

# plot max sharps ratio
fig.add_scatter(
    mode = 'markers',
    x = [temp.iloc[temp.sharpe_ratio.idxmax()]['port_vols']],
    y = [temp.iloc[temp.sharpe_ratio.idxmax()]['port_rets']],
    marker = dict(color='Red', size=20, symbol='star'),
    name = 'Max Sharpe'
).update(layout_showlegend=False)

# show spikes
fig.update_xaxes(showspikes=True)
fig.update_yaxes(showspikes=True)
fig.show()

In [37]:
import scipy.optimize as sco

In [38]:
sco.minimize?
#sco.minimize(
#    fun,
#    x0,
#    args=(),
#    method=None,
#    jac=None,
#    hess=None,
#    hessp=None,
#    bounds=None,
#    constraints=(),
#    tol=None,
#    callback=None,
#    options=None,
#)

In [39]:
def portfolio_stats(weights):
    
    weights = array(weights)[:,newaxis]
    port_rets = weights.T @ array(returns.mean()*252)[:,newaxis]
    port_vols = sqrt(multi_dot([weights.T, returns.cov()*252, weights] ))
    
    return array([port_rets, port_vols, port_rets/port_vols]).flatten()

In [40]:
# maximize sharpe ratio, only min func is available in scipy
# objective function
def min_sharpe_ratio(weights):
    return -portfolio_stats(weights)[2] # 2 to get sharpe ratio

In [41]:
# define initial weights
initial_wts = numofasset * [1/numofasset]
initial_wts

[0.2, 0.2, 0.2, 0.2, 0.2]

In [42]:
# each asset boundary ranges from 0 to 1
bnds = tuple((0,1) for x in range(numofasset))
bnds

((0, 1), (0, 1), (0, 1), (0, 1), (0, 1))

In [43]:
# specify constraints
cons = ({'type':'eq', 'fun': lambda x: sum(x)-1})
cons

{'type': 'eq', 'fun': <function __main__.<lambda>(x)>}

In [44]:
# optimizing for maximum sharpe ratio
opt_sharpe = sco.minimize(min_sharpe_ratio, initial_wts, method='SLSQP', bounds=bnds, constraints=cons)
opt_sharpe

     fun: -1.5019187754500378
     jac: array([ 2.98172235e-05,  6.83453977e-02,  4.21471789e-01, -1.47819519e-05,
        7.52508640e-06])
 message: 'Optimization terminated successfully'
    nfev: 36
     nit: 6
    njev: 6
  status: 0
 success: True
       x: array([0.21939285, 0.        , 0.        , 0.55733942, 0.22326773])

In [46]:
 # portfolio weights
list(zip(symbols, around(opt_sharpe['x']*100)))

[('AMD', 22.0), ('CSCO', 0.0), ('INTC', 0.0), ('INTU', 56.0), ('NVDA', 22.0)]

In [49]:
stats = ['Returns','Volatility','Sharpe Ratio']
list(zip(stats, around(portfolio_stats(opt_sharpe['x']),4)))

[('Returns', 0.5052), ('Volatility', 0.3364), ('Sharpe Ratio', 1.5019)]

In [52]:
# optimizing for minimum variance
# objective function
def min_variance(weights):
    return portfolio_stats(weights)[1]**2

In [54]:
# optimizing for minimum variance
opt_var = sco.minimize(min_variance, initial_wts, method='SLSQP', bounds=bnds, constraints=cons)
opt_var

     fun: 0.06349309772599004
     jac: array([0.12906377, 0.12698795, 0.12698044, 0.12698532, 0.14485935])
 message: 'Optimization terminated successfully'
    nfev: 60
     nit: 10
    njev: 10
  status: 0
 success: True
       x: array([0.        , 0.57709485, 0.13135064, 0.29155451, 0.        ])

In [55]:
# portfolio weights
list(zip(symbols, around(opt_var['x']*100,2)))

[('AMD', 0.0),
 ('CSCO', 57.71),
 ('INTC', 13.14),
 ('INTU', 29.16),
 ('NVDA', 0.0)]

In [56]:
stats = ['Returns','Volatility','Sharpe Ratio']
list(zip(stats, around(portfolio_stats(opt_var['x']),4)))

[('Returns', 0.2615), ('Volatility', 0.252), ('Sharpe Ratio', 1.0379)]

In [57]:
# optimizing to min volatility
def min_volatility(weights):
    return portfolio_stats(weights)[1]

In [62]:
# efficient frontier params
targetrets = linspace(0.24, 0.60, 100)
targetrets

array([0.24      , 0.24363636, 0.24727273, 0.25090909, 0.25454545,
       0.25818182, 0.26181818, 0.26545455, 0.26909091, 0.27272727,
       0.27636364, 0.28      , 0.28363636, 0.28727273, 0.29090909,
       0.29454545, 0.29818182, 0.30181818, 0.30545455, 0.30909091,
       0.31272727, 0.31636364, 0.32      , 0.32363636, 0.32727273,
       0.33090909, 0.33454545, 0.33818182, 0.34181818, 0.34545455,
       0.34909091, 0.35272727, 0.35636364, 0.36      , 0.36363636,
       0.36727273, 0.37090909, 0.37454545, 0.37818182, 0.38181818,
       0.38545455, 0.38909091, 0.39272727, 0.39636364, 0.4       ,
       0.40363636, 0.40727273, 0.41090909, 0.41454545, 0.41818182,
       0.42181818, 0.42545455, 0.42909091, 0.43272727, 0.43636364,
       0.44      , 0.44363636, 0.44727273, 0.45090909, 0.45454545,
       0.45818182, 0.46181818, 0.46545455, 0.46909091, 0.47272727,
       0.47636364, 0.48      , 0.48363636, 0.48727273, 0.49090909,
       0.49454545, 0.49818182, 0.50181818, 0.50545455, 0.50909

In [63]:
#targetrets - 25 = 0
#targetrets - 26 = 0
#targetrets - 27 = 0

In [66]:
tvols=[]
for tr in targetrets:
    
    ef_cons = ({'type': 'eq', 'fun': lambda x: portfolio_stats(x)[0] - tr}, # target return constraint
               {'type': 'eq', 'fun': lambda x: sum(x)-1})  # budgeting constraint
    
    opt_ef = sco.minimize(min_volatility, initial_wts, method='SLSQP', bounds=bnds, constraints=ef_cons)
    
    tvols.append(opt_ef['fun'])

    
targetvols = array(tvols)

In [67]:
efport = pd.DataFrame({
    'targetrets': around(100*targetrets, 2),
    'targetvols': around(100*targetvols, 2),
    'targetsharpe': around(targetrets/targetvols, 2)
})

In [68]:
efport.head()

,targetrets,targetvols,targetsharpe
0,24.00,25.33,0.95
1,24.36,25.29,0.96
2,24.73,25.25,0.98
3,25.09,25.23,0.99
4,25.45,25.21,1.01


In [73]:
# plot effecient frontier
fig = px.scatter(
    efport, x='targetvols', y='targetrets', color='targetsharpe',
    labels = {'targetrets': 'expected return', 'targetvols': 'expected volatility', 'targetsharpe': 'Sharpe Ratio'},
    title = "Effecient Frontier Portfolio"
).update_traces(mode='markers', marker=dict(symbol='cross'))

# plot max sharpe portfolio
fig.add_scatter(
    mode = 'markers',
    x = [100*portfolio_stats(opt_sharpe['x'])[1]], # vol
    y = [100*portfolio_stats(opt_sharpe['x'])[0]],
    marker = dict(color='red', size=20, symbol='star'),
    name = 'Max Sharpe'
).update(layout_showlegend=False)

# plot min variance portfolio
fig.add_scatter(
    mode = 'markers',
    x = [100*portfolio_stats(opt_var['x'])[1]],
    y = [100*portfolio_stats(opt_var['x'])[0]],
    marker = dict(color='red',size=20, symbol='star'),
    name='Max Sharpe'
).update(layout_showlegend=False)

# show spikes
fig.update_xaxes(showspikes=True)
fig.update_yaxes(showspikes=True)
fig.show()